In [ ]:
# Imports
import csv
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date


In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

target_date = date(2022, 2, 10)

preg_pi_screnning_ids = MaternalDataset.objects.filter(preg_pi=1,
                                   delivdt__lte=target_date - relativedelta(years=10)).values_list('screening_identifier')

consented_pi = SubjectConsent.objects.filter(screening_identifier__in=preg_pi_screnning_ids).values_list(
    'screening_identifier', flat=True)

locators = CaregiverLocator.objects.all().values_list('study_maternal_identifier', flat=True)

pi_dataset = MaternalDataset.objects.filter(
    preg_pi=1,).exclude(
        study_maternal_identifier__in=consented_pi).values_list(
        'study_maternal_identifier', flat=True)

assigned_pi = WorkList.objects.filter(study_maternal_identifier__in=pi_dataset,
                                      consented=False)

assigned_pi = [a for a in assigned_pi]

data = [['study_maternal_id', 'assigned', 'is_called', 'consented', 'phone_num_success', ], ]

ass_pi = []
call_att = []

for a in assigned_pi:
    log_entries = LogEntry.objects.filter(
        study_maternal_identifier=a.study_maternal_identifier).exclude(
            phone_num_success__in=['none_of_the_above']).order_by('created')
    x = [a.study_maternal_identifier, a.assigned, a.is_called, a.consented]

    phone_nums = []
    call_dates = []
    if log_entries:

        for g in log_entries:
            phone_nums.append(g.phone_num_success)
            call_dates.append(g.call_datetime.date().strftime('%Y-%m-%d'))
    x.append(phone_nums)
    x.append(call_dates)
    data.append(x)


In [ ]:
with open('/home/flourish/preg_pi_call_log.csv', 'w') as result_file:
    wr = csv.writer(result_file, dialect='excel')

    for d in data:
         wr.writerow(d)